In [1]:
# import modules
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
from abc import abstractmethod
from sklearn.datasets import load_digits
from sklearn.model_selection import KFold

# Base Classes

In [2]:
class Node:
    '''
      this class will later get the following attributes
      all nodes:
          features
          responses
      split nodes additionally:
          left
          right
          split_index
          threshold
      leaf nodes additionally
          prediction
    '''


class Tree:
    '''
      base class for RegressionTree and ClassificationTree
    '''
    def __init__(self, n_min=10):
        '''n_min: minimum required number of instances in leaf nodes
        '''
        self.n_min = n_min 

    def predict(self, x):
        ''' return the prediction for the given 1-D feature vector x
        '''
        # first find the leaf containing the 1-D feature vector x
        node = self.root
        while not hasattr(node, "prediction"):
            j = node.split_index
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        # finally, return the leaf's prediction
        return node.prediction

    def train(self, features, responses, D_try=None):
        '''
        features: the feature matrix of the training set
        response: the vector of responses
        '''
        N, D = features.shape
        assert(responses.shape[0] == N)

        if D_try is None:
            D_try = int(np.sqrt(D)) # number of features to consider for each split decision
        
        # initialize the root node
        self.root = Node()
        self.root.features  = features
        self.root.responses = responses

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            active_indices = self.select_active_indices(D, D_try)
            left, right = self.make_split_node(node, active_indices)
            if left is None: # no split found
                self.make_leaf_node(node)
            else:
                stack.append(left)
                stack.append(right)

    def make_split_node(self, node, indices):
        '''
        node: the node to be split
        indices: a numpy array of length 'D_try', containing the feature 
                         indices to be considered for the present split
                         
        return: None, None -- if no suitable split has been found, or
                left, right -- the children of the split
        '''
        # all responses equal => no improvement possible by any split
        if np.unique(node.responses).shape[0] == 1:
            return None, None
        
        # find best feature j_min (among 'indices') and best threshold t_min for the split
        l_min = float('inf')  # upper bound for the loss, later the loss of the best split
        j_min, t_min = None, None

        for j in indices:
            thresholds = self.find_thresholds(node, j)

            # compute loss for each threshold
            for t in thresholds:
                loss = self.compute_loss_for_split(node, j, t)

                # remember the best split so far 
                # (the condition is never True when loss = float('inf') )
                if loss < l_min:
                    l_min = loss
                    j_min = j
                    t_min = t

        if j_min is None: # no split found
            return None, None

        # create children for the best split
        left, right = self.make_children(node, j_min, t_min)

        # turn the current 'node' into a split node
        # (store children and split condition)
        node.left = left
        node.right = right
        node.threshold = t_min
        node.split_index = j_min

        # return the children (to be placed on the stack)
        return left, right

    def select_active_indices(self, D, D_try):
        ''' return a 1-D array with D_try randomly selected indices from 0...(D-1).
        '''
        return np.random.default_rng().choice(D, D_try, replace=False)

    def find_thresholds(self, node, j):
        ''' return: a 1-D array with all possible thresholds along feature j
        '''
        feature_values = node.features[:, j]
        return (feature_values[1:] + feature_values[:-1]) / 2

    def make_children(self, node, j, t):
        ''' execute the split in feature j at threshold t
        
            return: left, right -- the children of the split, with features and responses
                                   properly assigned according to the split
        '''
        left = Node()
        right = Node()

        left_children = node.features[:, j] <= t
        left.features = node.features[left_children]
        left.responses = node.responses[left_children]

        right.features = node.features[~left_children]
        right.responses = node.responses[~left_children]

        return left, right

    @abstractmethod
    def make_leaf_node(self, node):
        ''' Turn node into a leaf by computing and setting `node.prediction`
        
            (must be implemented in a subclass)
        '''
        raise NotImplementedError("make_leaf_node() must be implemented in a subclass.")

    @abstractmethod
    def compute_loss_for_split(self, node, j, t):
        ''' Return the resulting loss when the data are split along feature j at threshold t.
            If the split is not admissible, return float('inf').
        
            (must be implemented in a subclass)
        '''
        raise NotImplementedError("compute_loss_for_split() must be implemented in a subclass.")


# Regression Tree
To make the differentiation between a ClassificationTree and RegressionTree meaningful, we decided to use the mean of all responses as prediction for leaf nodes.

In [3]:
class RegressionTree(Tree):
    def __init__(self, n_min=10):
        super(RegressionTree, self).__init__(n_min)

    def compute_loss_for_split(self, node, j, t):
        # return the loss if we would split the instance along feature j at threshold t
        # or float('inf') if there is no feasible split
        left_children = node.features[:, j] <= t

        left_responses = node.responses[left_children]
        right_responses = node.responses[~left_children]

        if len(left_responses) < self.n_min or len(right_responses) < self.n_min:
            return float("inf")

        left_prediction = np.mean(left_responses)
        right_prediction = np.mean(right_responses)

        return np.sum((left_responses - left_prediction) ** 2) + np.sum((right_responses - right_prediction) ** 2)

    def make_leaf_node(self, node):
        # turn node into a leaf node by computing `node.prediction`
        # (note: the prediction of a regression tree is a real number)
        node.prediction = np.mean(node.responses)


# Classification Tree
We decided to use Gini Impurity as loss method, since it already perfectly supports multi-class trees out of the box.

In [4]:
class ClassificationTree(Tree):
    '''implement classification tree so that it can handle arbitrary many classes
    '''

    def __init__(self, classes, n_min=10):
        ''' classes: a 1-D array with the permitted class labels
            n_min: minimum required number of instances in leaf nodes
        '''
        super(ClassificationTree, self).__init__(n_min)
        self.classes = classes

    def compute_loss_for_split(self, node, j, t):
        # return the loss if we would split the instance along feature j at threshold t
        # or float('inf') if there is no feasible split
        left_children = node.features[:, j] <= t

        left_responses = node.responses[left_children]
        right_responses = node.responses[~left_children]

        if len(left_responses) < self.n_min or len(right_responses) < self.n_min:
            return float("inf")
        
        _, left_label_counts = np.unique(left_responses, return_counts=True)
        _, right_label_counts = np.unique(right_responses, return_counts=True)
        
        left_impurity = 1 - np.sum((left_label_counts / len(left_responses)) ** 2)
        right_impurity = 1 - np.sum((right_label_counts / len(right_responses)) ** 2)

        return left_impurity * len(left_responses) + right_impurity * len(right_responses)

    def make_leaf_node(self, node):
        # turn node into a leaf node by computing `node.prediction`
        # (note: the prediction of a classification tree is a class label)
        node.prediction = scipy.stats.mode(node.responses, axis=None)[0][0]


# Evaluation of Regression and Classification Tree

In [5]:
# read and prepare the digits data and extract 3s and 9s
digits = load_digits()
print(digits.data.shape, digits.target.shape)

instances = (digits.target == 3) | (digits.target == 9)
features = digits.data[instances, :]
labels = digits.target[instances]

# for regression, we use labels +1 and -1
responses = np.array([1 if l == 3 else -1 for l in labels])

assert(features.shape[0] == labels.shape[0] == responses.shape[0])

(1797, 64) (1797,)


In [16]:
# perform 5-fold cross-validation (see ex01) with responses +1 and -1 (for 3s and 9s)
# using RegressionTree()
# and comment on your results

splitter = KFold(n_splits=10)
n_mins = [1, 2, 5, 10, 20, 50]
classes = list(np.unique(labels))
grid = np.linspace(-10, 10, 100)

for n_min in n_mins:
    tree = RegressionTree(n_min=n_min)

    for train_index, test_index in splitter.split(features):
        tree.train(features[train_index], labels[train_index])
        deviations = np.zeros((len(classes), 2))

        for sample in test_index:
            label = labels[sample]
            prediction = tree.predict(features[sample])
            deviations[classes.index(label)] += [prediction, 1]

    print("Average scores for classes", classes, "with n_min =", n_min, "\n", deviations[:, 0] / deviations[:, 1])

Average scores for classes [3, 9] with n_min = 1 
 [4.         8.33333333]
Average scores for classes [3, 9] with n_min = 2 
 [3.83333333 8.16666667]
Average scores for classes [3, 9] with n_min = 5 
 [4.81388889 8.04166667]
Average scores for classes [3, 9] with n_min = 10 
 [3.47619048 8.27777778]
Average scores for classes [3, 9] with n_min = 20 
 [4.53015873 7.75238095]
Average scores for classes [3, 9] with n_min = 50 
 [4.53842242 7.87329109]


In [ ]:
# perform 5-fold cross-validation with labels 3 and 9
# using ClassificationTree(classes=np.unique(labels))
# and comment on your results

splitter = KFold(n_splits=10)
n_mins = [1, 2, 5, 10, 20, 50]
classes = list(np.unique(labels))

for n_min in n_mins:
    tree = ClassificationTree(classes=classes, n_min=n_min)
    confusion_matrix = np.zeros((len(classes), len(classes)))

    for train_index, test_index in splitter.split(features):
        tree.train(features[train_index], labels[train_index])

        for sample in test_index:
            label = labels[sample]
            prediction = tree.predict(features[sample])
            confusion_matrix[classes.index(label)][classes.index(prediction)] += 1

    print("Confusion matrix for classes", classes, "with n_min =", n_min, "\n", confusion_matrix)
    print("Accuracy: {:.1%}".format(np.trace(confusion_matrix) / np.sum(confusion_matrix)))

Because our training step took us less than one second per training set, and the variance between different runs caused by the random feature selection was too high to get meaningful results out of the cross-validation, we increased the number of splits to 10 and calculated the confusion matrices for different `n_min` values, all the way from 1 to 50.

Surprisingly, the impact of reducing `n_min` further below a threshold of approximately 10-20 does not seem to have any significant impact on the prediction quality, while still increasing the training duration (Due to the higher depth of the resulting tree). Overall, a single ClassificationTree was only able to reach an accuracy of 85-90%.

## Proof of Concept: Multi-Class Classification
Our current implementations of RegressionTree and ClassificationTree support multiple classes:

RegressionTree can handle any score (Not just -1 and 1), as already shown in the cross-valdation.
To demonstrate the multi-class functionality of ClassificationTree, we will run a simple test case with three instead of two labels.

Technically speaking, we even ignore the supplies `classes` parameter, and only carry it for compatibility purposes.
The actually returned label in ClassificationTree is generated directly from the `reponses` supplied during training, which is why the number of labels don't matter to our implementation.

In [19]:
instances = (digits.target == 3) | (digits.target == 7) | (digits.target == 9)
features = digits.data[instances, :]
labels = digits.target[instances]

splitter = KFold(n_splits=10)
classes = list(np.unique(labels))

tree = ClassificationTree(classes=classes)
confusion_matrix = np.zeros((len(classes), len(classes)))

for train_index, test_index in splitter.split(features):
    tree.train(features[train_index], labels[train_index])

    for sample in test_index:
        label = labels[sample]
        prediction = tree.predict(features[sample])
        confusion_matrix[classes.index(label)][classes.index(prediction)] += 1

print("Confusion matrix for classes", classes, "\n", confusion_matrix)
print("Accuracy: {:.1%}".format(np.trace(confusion_matrix) / np.sum(confusion_matrix)))

Confusion matrix for classes [3, 7, 9] 
 [[155.   6.  22.]
 [  8. 148.  23.]
 [ 31.  13. 136.]]
Accuracy: 81.0%


# Regression and Classification Forest

In [ ]:
def bootstrap_sampling(features, responses):
    '''return a bootstrap sample of features and responses
    '''
    ... # your code here
    raise NotImplementedError("bootstrap_sampling(): remove this exception after adding your code above.")

In [ ]:
class RegressionForest():
    def __init__(self, n_trees, n_min=10):
        # create ensemble
        self.trees = [RegressionTree(n_min) for i in range(n_trees)]
    
    def train(self, features, responses):
        for tree in self.trees:
            boostrap_features, bootstrap_responses = bootstrap_sampling(features, responses)
            tree.train(boostrap_features, bootstrap_responses)

    def predict(self, x):
        # compute the response of the ensemble from the individual responses and return it
        ... # your code here
        raise NotImplementedError("predict(): remove this exception after adding your code above.")

In [ ]:
class ClassificationForest():
    def __init__(self, n_trees, classes, n_min=1):
        self.trees = [ClassificationTree(classes, n_min) for i in range(n_trees)]
        self.classes = classes
    
    def train(self, features, responses):
        for tree in self.trees:
            boostrap_features, bootstrap_responses = bootstrap_sampling(features, responses)
            tree.train(boostrap_features, bootstrap_responses)

    def predict(self, x):
        # compute the response of the ensemble from the individual responses and return it
        ... # your code here
        raise NotImplementedError("predict(): remove this exception after adding your code above.")

# Evaluation of Regression and Decision Forest

In [ ]:
# perform 5-fold cross-validation (see ex01) with responses +1 and -1 (for 3s and 9s)
# using RegressionForest(n_trees=10)
# and comment on your results
... # your code here

In [ ]:
# perform 5-fold cross-validation with labels 3 and 9
# using DecisionForest(n_trees=10, classes=np.unique(labels))
# and comment on your results
... # your code here

# Multi-class Classification Forest

In [ ]:
# Train DecisionForest(n_trees=10, classes=np.unique(digits.target))
# for all 10 digits simultaneously.
# Compute and plot the confusion matrix after 5-fold cross-validation and comment on your results.
... # your code here

# One-against-the-rest classification with RegressionForest

In [ ]:
# Train ten one-against-the-rest regression forests for the 10 digits.
# Make sure that all training sets are balanced between the current digit and the rest.
# Assign test instances to the digit with highest score, 
# or to "unknown" if all scores are negative.
# Compute and plot the confusion matrix after 5-fold cross-validation and comment on your results.
... # your code here